# NiiVue Loading Examples

This example shows 4 different methods to load neuroimaging data into ipyniivue:

1. load from local path
2. download from URL and then load
3. load from URL
4. load from name + raw binary data

For data from URLs, be aware that Browsers enforce CORS (Cross-Origin Resource Sharing), so ipyniivue can only load data directly from URLs if the remote server allows it. 

There's an additional extra example at the end that shows modifying raw data and loading it into ipyniivue.

In [ ]:
import ipyniivue
import numpy as np
import pathlib

try:
    import nibabel as nib
except ModuleNotFoundError:
    !pip install nibabel
    import nibabel as nib

## Load from Local Path

This example loads a dataset that is already available on the local file system.

In [ ]:
nv1 = ipyniivue.NiiVue()
nv1.load_volumes([{'path': '../images/mni152.nii.gz'}])
nv1

# Download from URL and then Load from Path

Here we fetch data from the web and subsequently load the resulting image.

In [ ]:
nv2 = ipyniivue.NiiVue()

github_url = "https://github.com/niivue/niivue-demo-images/raw/main"

DATA_FOLDER = pathlib.Path("images")

ipyniivue.download_dataset(
    api_url=github_url,
    dest_folder=DATA_FOLDER,
    files=[
        "mni152.nii.gz",
    ],
)

nv2.load_volumes([{"path": DATA_FOLDER / "mni152.nii.gz"}])
nv2

## Load from URL
Direct loading from web URL

In [ ]:
nv3 = ipyniivue.NiiVue()

nv3.load_volumes([{"url": "https://niivue.com/demos/images/mni152.nii.gz"}])
nv3

## Load from Binary Data

This example generates an image in memory, and loads the image directly without saving it to the file system.

In [ ]:
nv4 = ipyniivue.NiiVue()

data = np.zeros((64, 64, 64), dtype=np.float32)
center = np.array([32, 32, 32])
for i in range(64):
    for j in range(64):
        for k in range(64):
            dist = np.sqrt((i-center[0])**2 + (j-center[1])**2 + (k-center[2])**2)
            if dist < 20:
                data[i, j, k] = 1.0 - (dist / 20.0)

affine = np.eye(4)
nifti = nib.Nifti1Image(data, affine)

nifti_bytes = nifti.to_bytes()

volume = ipyniivue.Volume(name="sphere.nii", data=nifti_bytes)
nv4.load_volumes([volume])
nv4

## Extra example

Load with nibabel, modify by adding vertical rod, then pass binary data to ipyniivue

In [ ]:
nv5 = ipyniivue.NiiVue()

img = nib.load(DATA_FOLDER / "mni152.nii.gz")
data = img.get_fdata()

max_val = data.max()
x_center = data.shape[0] // 2
y_center = data.shape[1] // 2

for z in range(20, data.shape[2] - 20):
    for dx in range(-3, 4):
        for dy in range(-3, 4):
            if dx*dx + dy*dy <= 9:
                data[x_center + dx, y_center + dy, z] = max_val * 2

modified_nifti = nib.Nifti1Image(data.astype(np.float32), img.affine, img.header)

nifti_bytes = modified_nifti.to_bytes()

volume = ipyniivue.Volume(name="modified_mni.nii", data=nifti_bytes)
nv5.load_volumes([volume])
nv5